# GDVSpectra Likelihood module tests

## Globals

In [21]:
# globals (dev)
FOLDER_MODULES = r'C:\Users\262272G\Documents\GitHub\tenement-tools\modules'  
FOLDER_SHARED = r'C:\Users\262272G\Documents\GitHub\tenement-tools\shared'
TEST_MODULE = r'C:\Users\262272G\Documents\GitHub\tenement-tools\tests\code'
GRP_LYR_FILE = r'C:\Users\262272G\Documents\GitHub\tenement-tools\arc\lyr\group_template.lyrx'    

## Setup

### Imports

In [22]:
# imports
import os
import random
import numpy as np
import xarray as xr

# import testing functions
sys.path.append(TEST_MODULE)
import test_funcs

# import toolbox
arcpy.ImportToolbox(r"C:\Users\262272G\Documents\GitHub\tenement-tools\arc\toolbox\tenement-tools-toolbox.pyt")

<module 'toolbox'>

### Reload libraries

In [23]:
from importlib import reload
reload(test_funcs)

<module 'test_funcs' from 'C:\\Users\\262272G\\Documents\\GitHub\\tenement-tools\\tests\\code\\test_funcs.py'>

### Set data files and locations

In [24]:
# setup general io
#input_folder = r'E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs'
#output_folder = r'E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\outputs'
input_folder = r'C:\Users\262272G\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs'
output_folder = r'C:\Users\262272G\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\outputs'
temp_nc = os.path.join(input_folder, 'temp_nc.nc')  # temp nc file for use when breaking ncs

# setup landsat cubes paths
ls_cubes = [
    #r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\yandi_1_ls_90_20_raw_odc.nc",
    #r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\yandi_2_ls_90_20_raw_odc.nc",
    #r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\yandi_3_ls_90_20_raw_odc.nc",
    #r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\yandi_4_ls_90_20_raw_odc.nc",
    
    r"C:\Users\262272G\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\yandi_2_ls_90_20_raw_odc.nc",
]

# setup sentinel2 cubes paths
s2_cubes = [
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\yandi_1_s2_16_20_raw_odc.nc",
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\yandi_2_s2_16_20_raw_odc.nc",
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\yandi_3_s2_16_20_raw_odc.nc",
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\yandi_4_s2_16_20_raw_odc.nc",
]

### Setup dataset corruptor functions

In [25]:
# these dict items contain a dataset corruptor function and associated params
# comment any out that aren't required.
def build_data_corruptors():
    
    # set up list
    nc_corruptors = []

    # func: raw default dataset, no changes 
    nc_corruptors.append({'func': test_funcs.default, 'in_nc': temp_nc})

    # func: remove x, y, time, spatial_ref coords only (note: seperate tests)
    nc_corruptors.append({'func': test_funcs.remove_coord, 'in_nc': temp_nc, 'coord': 'x'})
    nc_corruptors.append({'func': test_funcs.remove_coord, 'in_nc': temp_nc, 'coord': 'y'})
    nc_corruptors.append({'func': test_funcs.remove_coord, 'in_nc': temp_nc, 'coord': 'time'})
    nc_corruptors.append({'func': test_funcs.remove_coord, 'in_nc': temp_nc, 'coord': 'spatial_ref'})

    # func: remove red and oa_fmask band vars (note: seperate tests)
    nc_corruptors.append({'func': test_funcs.remove_var, 'in_nc': temp_nc, 'var': 'nbart_red'})
    nc_corruptors.append({'func': test_funcs.remove_var, 'in_nc': temp_nc, 'var': 'oa_fmask'})

    # func: set all vars to nan
    nc_corruptors.append({'func': test_funcs.set_nc_vars_all_nan, 'in_nc': temp_nc})

    # func: set all vars for 10 rand times to all nan
    nc_corruptors.append({'func': test_funcs.set_nc_vars_random_all_nan, 'in_nc': temp_nc, 'num': 10})

    # func: strip all attrs from nc    
    nc_corruptors.append({'func': test_funcs.strip_nc_attributes, 'in_nc': temp_nc})

    # func: set vars in first and last time index to all nan
    nc_corruptors.append({'func': test_funcs.set_end_times_to_all_nan, 'in_nc': temp_nc})

    # func: reduce whole nc to one random time slice
    nc_corruptors.append({'func': test_funcs.reduce_to_one_scene, 'in_nc': temp_nc})

    # func: set wet months all nan, all years, for specific months (note: seperate tests)
    nc_corruptors.append({'func': test_funcs.set_all_specific_season_nan, 'in_nc': temp_nc, 'months': [1]})
    nc_corruptors.append({'func': test_funcs.set_all_specific_season_nan, 'in_nc': temp_nc, 'months': [1, 2, 3]})
    nc_corruptors.append({'func': test_funcs.set_all_specific_season_nan, 'in_nc': temp_nc, 'months': [7, 8, 9, 10, 11, 12]})

    # func: set wet months all nan, specific years, for specific months (note: seperate tests)
    nc_corruptors.append({'func': test_funcs.set_specific_years_season_nan, 'in_nc': temp_nc, 'years': [1990], 'months': [1, 2, 3]})
    nc_corruptors.append({'func': test_funcs.set_specific_years_season_nan, 'in_nc': temp_nc, 'years': [2005], 'months': [1, 2, 3]})
    nc_corruptors.append({'func': test_funcs.set_specific_years_season_nan, 'in_nc': temp_nc, 'years': [2006, 2007], 'months': [1, 2, 3]})

    # func: drop wet months, all years, for specific months (note: seperate tests)
    nc_corruptors.append({'func': test_funcs.remove_all_specific_season_nan, 'in_nc': temp_nc, 'months': [1]})
    nc_corruptors.append({'func': test_funcs.remove_all_specific_season_nan, 'in_nc': temp_nc, 'months': [1, 2, 3]})
    nc_corruptors.append({'func': test_funcs.remove_all_specific_season_nan, 'in_nc': temp_nc, 'months': [7, 8, 9, 10, 11, 12]})

    # func: drop wet months, specific years, for specific months (note: seperate tests)
    nc_corruptors.append({'func': test_funcs.remove_specific_years_season_nan, 'in_nc': temp_nc, 'years': [1990], 'months': [1, 2, 3]})
    nc_corruptors.append({'func': test_funcs.remove_specific_years_season_nan, 'in_nc': temp_nc, 'years': [2009], 'months': [1, 2, 3]})
    nc_corruptors.append({'func': test_funcs.remove_specific_years_season_nan, 'in_nc': temp_nc, 'years': [2007, 2008], 'months': [1, 2, 3]})
    
    return nc_corruptors

nc_corruptors = build_data_corruptors()

In [5]:
# inputs

# veg and mst index
# try only veg
# try only mst
# try unavailble veg
# try unavalble mst
# try all veg
# try all mst

# outlier correction
# zscore value is wrong
# 0.01, 0.1, 0.05
# try none
# try 0
# try no cube data
# try dims

# invariant standardisation
# upper 0.99, lower 0.05
# try none for upper and lower
# try 0 for both
# try - values

## Run tests

### Test One: Wet Months

In [10]:
def _test_one_wet_months(in_nc, out_func_name='default'):
    """tests different set ups of the wet months input"""

    # set default params for tool
    inputs = {
        'in_nc': in_nc,                        # input nc (i.e. temp nc)
        'out_nc': '',                          # output nc (i.e. t1a nc)
        'in_wet_months': '',                   # wet months 
        'in_dry_months': '9;10;11',            # dry months 
        'in_veg_idx': 'MAVI',                  # vege index name
        'in_mst_idx': 'NDMI',                  # moisture index name       
        'in_zscore_pvalue': None,              # zscore pvalue
        'in_ivt_qupper': 0.99,                 # upper quantile for standardisation
        'in_ivt_qlower': 0.05,                 # lower quantile for standardisation
        'in_fmask_flags': 'Valid;Snow;Water',  # fmask flag values
        'in_max_cloud': 10,                    # max cloud percentage
        'in_interpolate': True,                # interpolate missing pixels
        'in_add_result_to_map': True,          # add result to map
    }
    
    # set output test string convention
    out_nc = os.path.join(output_folder, 't1{}_{}.nc')
    
    try:
        print('\n\nRunning t1a (test one a). Wet months input is "".')
        print('- ' * 50)
        inputs.update({'out_nc': out_nc.format('a', out_func_name), 'in_wet_months': ''})
        arcpy.GDVSpectra_Likelihood_toolbox(*list(inputs.values()))
    except Exception as e:
        print(e)
        
    try:
        _ = str(random.randint(1, 5))
        print('\n\nRunning t1b (test one b). Wet months have 1 month only ({}).'.format(_))
        print('- ' * 50)
        inputs.update({'out_nc': out_nc.format('b', out_func_name), 'in_wet_months': _})
        arcpy.GDVSpectra_Likelihood_toolbox(*list(inputs.values()))
    except Exception as e:
        print(e)      
        
    try:
        print('\n\nRunning t1c (test one c). Wet months have several months (1, 2, 3).')
        print('- ' * 50)
        inputs.update({'out_nc': out_nc.format('c', out_func_name), 'in_wet_months': '1;2;3'})
        arcpy.GDVSpectra_Likelihood_toolbox(*list(inputs.values()))
    except Exception as e:
        print(e)   
        
    try:
        print('\n\nRunning t1d (test one d). Wet months contains months in dry (5, 6, 7, 8, 9).')
        print('- ' * 50)
        inputs.update({'out_nc': out_nc.format('d', out_func_name), 'in_wet_months': '5;6;7;8;9'})
        arcpy.GDVSpectra_Likelihood_toolbox(*list(inputs.values()))
    except Exception as e:
        print(e)  
        
    try:
        print('\n\nRunning t1e (test one e). Wet months contains 1 month in december (12, 1, 2).')
        print('- ' * 50)
        inputs.update({'out_nc': out_nc.format('e', out_func_name), 'in_wet_months': '12;1;2'})
        arcpy.GDVSpectra_Likelihood_toolbox(*list(inputs.values()))
    except Exception as e:
        print(e)
        

def perform_tests(in_nc, temp_nc, nc_corruption_funcs):
    """iterates corrupt ncs and runs through parameter tests (e.g. wet season months)"""
    
    print('\nRunning all data corruption functions through test one (wet season months).')
    print('- ' * 50)

    # iterate different dataset corruptors and feed each into test func: one wet months
    for func in nc_corruption_funcs:

        # get func name and notify
        print('\nRunning data corruption function: {}'.format(func['func'].__name__))
        print('- ' * 50)
        help(func['func'])

        # make new instance of func dict so we dont destroy keys
        func_copy = func.copy()

        # get parameter names and entered value
        params, inputs = [], []
        for k, v in func_copy.items():
            if k != 'func':
                print('With parameter: {} of value: {}'.format(k, v))

        # start the test
        print('\nBeginning test.')

        # duplicate raw netcdf for testing
        test_funcs.create_temp_nc(in_nc=in_nc, out_nc=temp_nc)

        # prepare current function with associated parameters and run
        f = func_copy['func']  # get func only
        func_copy.pop('func')  # get params only
        f(**func_copy)         # run func

        # test current corrupted netcdf with various wet month scenarios
        _test_one_wet_months(temp_nc, out_func_name=str(func['func'].__name__))

        # add some space to print out
        print('- ' * 50)
        print('\n\n\n')

In [ ]:
# set specific dataset
nc_file = ls_cubes[0]
perform_tests(in_nc=nc_file, temp_nc=temp_nc, data_corruption_funcs)

### Landsat

In [ ]:
# inputs
#  sat cube from cog
# try dims
# try no crs
# try no data
# try missing attrs
# try 1 image
# try all nan
# try some nan
# try very large ds
# try very small ds
# try wrong bands

In [ ]:
# set testing parameters. these parameters must match arcgis pro input
# syntax (i.e. values brought in directly from pro interface controls)

# default
test_1 = {
    'in_nc': r'C:\Users\Lewis\Desktop\ryan\ls.nc',        # raw input satellite netcdf
    'out_nc': r'C:\Users\Lewis\Desktop\ryan\ls_like.nc',  # raw output likelihood netcdf    
    'in_wet_months': '1;2;3',                             # wet months 
    'in_dry_months': '9;10;11',                           # dry months 
    'in_veg_idx': 'MAVI',                                 # vege index name
    'in_mst_idx': 'NDMI',                                 # moisture index name       
    'in_zscore_pvalue': None,                             # zscore pvalue
    'in_ivt_qupper': 0.99,                                # upper quantile for standardisation
    'in_ivt_qlower': 0.05,                                # lower quantile for standardisation
    'in_fmask_flags': 'Valid;Snow;Water',                 # fmask flag values
    'in_max_cloud': 10,                                   # max cloud percentage
    'in_interpolate': True,                               # interpolate missing pixels
    'in_add_result_to_map': True,                         # add result to map
}

#execute(**test_1)

### Sentinel

In [ ]:
# default
test_2 = {
    'in_nc': r'C:\Users\Lewis\Desktop\ryan\ls.nc',        # raw input satellite netcdf
    'out_nc': r'C:\Users\Lewis\Desktop\ryan\ls_like.nc',  # raw output likelihood netcdf    
    'in_wet_months': '1;2;3',                             # wet months 
    'in_dry_months': '9;10;11',                           # dry months 
    'in_veg_idx': 'MAVI',                                 # vege index name
    'in_mst_idx': 'NDMI',                                 # moisture index name       
    'in_zscore_pvalue': None,                             # zscore pvalue
    'in_ivt_qupper': 0.99,                                # upper quantile for standardisation
    'in_ivt_qlower': 0.05,                                # lower quantile for standardisation
    'in_fmask_flags': 'Valid;Snow;Water',                 # fmask flag values
    'in_max_cloud': 10,                                   # max cloud percentage
    'in_interpolate': True,                               # interpolate missing pixels
    'in_add_result_to_map': True,                         # add result to map
}